# Initialize here!

In [ ]:
import random
import pandas as pd
from pandas import Series, DataFrame
import nbinteract as nbi
from IPython.display import clear_output # https://stackoverflow.com/questions/24816237/ipython-notebook-clear-cell-output-in-code/24818304


def bulletCalculation (bullet, STR, DEX, cover, distance, enemyGun_Armor): 
    bulletList = []
    successList = []
    aggro_Counter = 0
    hit_Counter = 0
    close_Miss_Counter = 0
    shotsFiredCounter = 0
    shotsFired = bullet
    
    bullet = bullet * weaponSpread
    
    for shot in range (1, bullet + 1):
        
        # Spread weapons can shoot multiple projectiles without recoil.
        if weaponType == "1" and shot <= weaponSpread:
            # First shot and first spread has no recoil.
            recoilCalc = 0
        elif weaponType == "1" and (shot / (weaponSpread + 1)) == (1 + shotsFiredCounter) :
            shotsFiredCounter += 1
            recoilCalc = ((shotsFiredCounter) * (recoil / 12) / (STR / 25)) ** 3 / 100
        elif weaponType == "0":
            # Default recoil if not spread weapon.
            recoilCalc = ((shot - 1) * (recoil / 12) / (STR / 25)) ** 2 / 100
            
        bullet_ACC = 1 - recoilCalc
        
        if bullet_ACC <= 0:
            aggroFiring = .1 - (.01 * aggro_Counter ** 2) 
            aggro_Counter += 1
            bullet_ACC = aggroFiring
        if aimed == True:
            # If the shot is aimed, 1.5x bonus to dexterity.
            DEX_Base = (DEX / 16 * accuracy / 12)
        else:
            DEX_Base = (DEX / 25 * accuracy / 12)
            
        if cover == 1:
            cover = 10
        elif cover == 2:
            cover = 50
        elif cover == 3:
            cover = 70
        elif cover == 4:
            cover = 90
        elif cover == 5:
            cover = 99
        # else: Broken 
        #    print ("Invalid Cover Input. Defaulting to 0.")
        #    cover = 0
        
        if cover == 0:
            bullet_Score = int((bullet_ACC * DEX_Base) * 1000)
            bullet_Score = distanceModifier (bullet_Score, distance) * 2
        else:    
            # Have cover get affected by playerGun_Piercing?
            bullet_Score = round( (bullet_ACC * DEX_Base) * 1000 )
            bullet_Score = distanceModifier (bullet_Score, distance) * ((100 
            - cover) / 100)
            
        bullet_Score = distanceModifier (bullet_Score, distance)
        bulletList.append (bullet_Score)

    completely_Miss = complete_Miss_Calc (bulletList)
    

    successList = hit_Indicator (bulletList)
    for shot in successList:
        if shot == "Hit!":
            hit_Counter += 1
        elif shot == "Close Miss!":
            close_Miss_Counter += 1

    
    phy_DMG_Dic = gun_PHY_DMG (enemyGun_Armor, hit_Counter)
    men_DMG = gun_MEN_DMG (enemyGun_Armor, hit_Counter, close_Miss_Counter)
    
    phy_DMG = phy_DMG_Dic["Physical DMG"]
    DMG_CHG = phy_DMG_Dic["DMG Modified"]

    
    print (bulletList)   
    print ("You shoot", shotsFired, "times.", "You managed to hit your target", hit_Counter, "times", "out of",
           bullet, "times")
    print ("You have a " + str(completely_Miss) + "% chance of completely "
           "missing this turn.")
    print (phy_DMG, "PHY DMG", men_DMG, "MEN DMG." )
    
    if DMG_CHG > 0:
        print ("+" + str(DMG_CHG) + " PHY DMG "+ "from Piercing!")
    elif DMG_CHG < 0:
        print (str(DMG_CHG) + " PHY DMG " +"from Armor!")
    print ("")
    # print ("YOU ARE", charaDF.iloc[CHAR_LOAD_NUM][0]) It seems like .iloc refers to order purely, not name. If an index is
    # set to 2, and it is the only row in the DF, it is considered a 0 by .iloc. 2 is a name according to it despite being a
    # number. So, use .loc instead to refer to the specific name. 
    # .iloc is for PURE ORDERING
    print ("YOU ARE", charaDF.loc[CHAR_LOAD_NUM]["CHAR NAME"].upper())
    print ("YOUR GUN IS", gunDF.loc[GUN_LOAD_NUM]["GUN NAME"].upper())
    
def distanceModifier (bullet_Score, distance):
    if distance == 0:
        distance = 10
    elif distance == 1:
        distance = 25
    elif distance == 2:
        distance = 50
        
    modified_Score = int(bullet_Score * (1 - (distance / playerGun_Range)))
    return modified_Score
    
def hit_Indicator (bulletList):
    successList = []
    for bullet_Chance in bulletList:
        roll = random.randint(1, 1000)
        
        if roll < bullet_Chance:
            successList.append("Hit!")
        elif (bullet_Chance - roll) < 0 and (bullet_Chance - roll) > -100:
            successList.append("Close Miss!")
        else:
            successList.append("Miss!")
    print (successList)
    return successList
    
    
def complete_Miss_Calc (bulletList):
    completely_Miss = 1
    
    for holder in bulletList:
        if holder <= 0:
            miss = 1
        else:
            miss = (1000 - holder) / 1000
        completely_Miss *= miss
    completely_Miss = round(completely_Miss * 100)
    
    if completely_Miss < 0:
        completely_Miss = 0
    return completely_Miss

def fireRate (bullet):
    global aimed
    aimed = False
    
    if bullet == "0":
        aimed = True
        return 1
    elif bullet == "1":
        return (random.randint(2, 3)) 
    elif bullet == "3":
        return random.randint(round(.5 * rateOfFire), rateOfFire) 
    else:
        # If not entered, controlled fire. I may have conservative and controlled
        # take into account actual accuracy per shot before ceasing. Depends.
        # Anything less than 8 ROF is just 1 ROF. Can only used aimed shots.
        return random.randint(4, round(.5 * rateOfFire))
 
def createGunStats ():
    # Add in range, damage, and AP. Range for average AR is 800 in this system so far.
    # Use 2x or 2.5x effective range. Tec - 9 is 50 so 100? 1911 is 100 so 200?.
    
    global accuracy
    global recoil
    global rateOfFire
    global gunMin
    global gunMax
    global playerGun_Piercing
    global playerGun_Range
    global weaponType
    global weaponSpread # If explosive, reroll for each person in vicinity.
    global GUN_LOAD_NUM
    
    init = 1
    
    keepActive = "1"
    while keepActive == "1":
        if checkGunDB == "1":
            print (gunDF)
        print ("")
        print ("To overwrite a gun, insert in same load number!")
        GUN_LOAD_NUM = int(input("Where do you want to save? [ALWAYS Start with 0!]:" ))
        
        GUN_NAME = input("Enter Gun Name: ")
        
        print ("Gun Stats")
        accuracy = int(input ("Accuracy?: "))
        recoil = int(input ("Recoil?: "))
        rateOfFire = int(input ("Fire Rate Per Standard Act (AR: 20, Pistol: 10): "))
        gunMin = int(input ("Min DMG?: "))
        gunMax = int(input ("Max DMG?: "))
        playerGun_Piercing = int(input ("Piercing?: "))
        weaponType = input("1 for spread/explosive weapons. Blank otherwise: ")
        
        if weaponType == "":
            weaponType = "0"
            
        if weaponType == "1":
            weaponSpread = int(input("What is the spread of the weapon?: "))
        else:
            weaponSpread = 1
        
        # Difference in playerGun_Piercing and armor of 5 should do no damage?
        # Need to maybe eliminate extra damage from playerGun_Piercing? Easier for balance?
        # Piercing may need some update.
        playerGun_Range = int(input ("Gun Range? (AR: 800, Pistol: 200): "))
        gunDB(GUN_NAME, init, GUN_LOAD_NUM)
        if init == 1:
            init = 0
        if len(gunDF) > 0 and checkGunDB != "1":
            print(gunDF)
        keepActive = input("If you wish to create another gun, press 1. Otherwise, enter any key.")
        clear_output()
        print (gunDF)
        
def loadGunStats():
    global accuracy
    global recoil
    global rateOfFire
    global gunMin
    global gunMax
    global playerGun_Piercing
    global playerGun_Range
    global weaponType
    global weaponSpread # If explosive, reroll for each person in vicinity.
    global GUN_LOAD_NUM

    # https://stackoverflow.com/questions/743164/emulate-a-do-while-loop-in-python
    print (gunDF)
    GUN_LOAD_NUM = int(input("Insert Load Number: "))
    accuracy = gunDF.iloc[GUN_LOAD_NUM]["accuracy"]
    recoil = gunDF.iloc[GUN_LOAD_NUM]["recoil"]
    rateOfFire = gunDF.iloc[GUN_LOAD_NUM]["ROF"]
    gunMin = gunDF.iloc[GUN_LOAD_NUM]["Dmg Min"]
    gunMax = gunDF.iloc[GUN_LOAD_NUM]["Dmg Max"]
    playerGun_Piercing = gunDF.iloc[GUN_LOAD_NUM]["Piercing"]
    playerGun_Range = gunDF.iloc[GUN_LOAD_NUM]["Range"]
    weaponType = gunDF.iloc[GUN_LOAD_NUM]["Type"]
    weaponSpread = gunDF.iloc[GUN_LOAD_NUM]["Spread"]
    return
    
def createShooterStats():
    global DEX
    global STR
    global eff_DEX
    global eff_STR
    global GUN_TRAIN
    global eff_GUN_TRAIN
    global CHAR_LOAD_NUM
    init = 1
    
    keepActive = "1"
    while keepActive == "1":
        if checkCharaDB == "1":
            print (charaDF)

        print ("")
        print ("To overwrite a character, insert in same load number!")
        CHAR_LOAD_NUM = int(input("Where do you want to save? [ALWAYS Start with 0!]:" ))
        CHAR_NAME = input("Enter Character Name: ")
        print ("Enter your character's stats")
        
        GUN_TRAIN = (input ("What is training level 0 - 4? (Blank or 2 is Sufficiently"
        + " Trained or Experienced): "))
        if GUN_TRAIN == "0":
            GUN_TRAIN = 0
            eff_GUN_TRAIN = .5
        elif GUN_TRAIN == "1":
            GUN_TRAIN = 1
            eff_GUN_TRAIN = .8
        elif GUN_TRAIN == "2":
            GUN_TRAIN = 2
            eff_GUN_TRAIN = 1
        elif GUN_TRAIN == "3":
            GUN_TRAIN = 3
            eff_GUN_TRAIN = 1.15
        elif GUN_TRAIN == "4":
            GUN_TRAIN = 4
            eff_GUN_TRAIN = 1.3
        else:
            GUN_TRAIN = 1
            
        DEX = int(input ("What is dexterity?: ")) 
        STR = int(input ("What is strength?: ")) 
        eff_DEX = DEX * eff_GUN_TRAIN
        eff_STR = STR * eff_GUN_TRAIN
        

        charaDB(DEX, STR, eff_DEX, eff_STR, GUN_TRAIN, eff_GUN_TRAIN, CHAR_NAME, init, CHAR_LOAD_NUM)
        if init == 1:
            init = 0
        
            
        keepActive = input("If you wish to create another character, press 1. Otherwise, enter any key.")
        clear_output()
        if len(charaDF) > 0 and checkCharaDB != "1":
            print(charaDF)

    
    # Hates rate of fire 1. Command line hates it. Okay here...
    #Appears to me that bullet = nothing and was being plugged 

def loadShooterStats():
    global DEX
    global STR
    global eff_DEX
    global eff_STR
    global GUN_TRAIN
    global eff_GUN_TRAIN
    global CHAR_LOAD_NUM
  # https://stackoverflow.com/questions/743164/emulate-a-do-while-loop-in-python
    print (charaDF)
    CHAR_LOAD_NUM = int(input("Insert Load Number: "))
    eff_DEX = charaDF.iloc[CHAR_LOAD_NUM]["EFF DEX"]
    eff_STR = charaDF.iloc[CHAR_LOAD_NUM]["EFF STR"]
    eff_GUN_TRAIN = charaDF.iloc[CHAR_LOAD_NUM]["EFF GUN TRAIN"]
    DEX = charaDF.iloc[CHAR_LOAD_NUM]["DEX"]
    STR = charaDF.iloc[CHAR_LOAD_NUM]["STR"]
    GUN_TRAIN = charaDF.iloc[CHAR_LOAD_NUM]["GUN TRAIN"]
    return

def gun_PHY_DMG (enemyGun_Armor, hit_Counter):
    # Do I do flat damage modified by how well it was hit?
    # Or do I assume the damage handles how well it was hit like this one?
    # Note: Make armor/playerGun_Piercing calculation separate, so we can see how it increases/decreases damage.
    # Having playerGun_Piercing get affected by range.
    singlePhy_DMG = 0
    totalPhy_DMG = 0
    DMG_CHG = 0
    armorMod = ((1 + enemyGun_Armor / 10) - (playerGun_Piercing / 10))
    
    for phy_Attempt in range (0, hit_Counter):
        noMod_DMG = random.randint(gunMin, gunMax)
        singlePhy_DMG = noMod_DMG / armorMod
        totalPhy_DMG += singlePhy_DMG
        DMG_CHG += singlePhy_DMG - noMod_DMG
            
    phy_DMG_List = {"Physical DMG" : round(totalPhy_DMG), "DMG Modified" : round(DMG_CHG)}
    
    return phy_DMG_List

def charaDB (DEX, STR, eff_DEX, eff_STR, GUN_TRAIN, eff_GUN_TRAIN, CHAR_NAME, init, CHAR_LOAD_NUM = 0):
    global charaDF
    
    if init == 1:
        charaDic = {"CHAR NAME": CHAR_NAME, "DEX": DEX, "STR": STR, "EFF DEX": eff_DEX, "EFF STR": eff_STR,
                    "GUN TRAIN": GUN_TRAIN, "EFF GUN TRAIN": eff_GUN_TRAIN, "CHARA LOAD NUM": CHAR_LOAD_NUM}
        charaDF = pd.DataFrame(charaDic, index = [0])
        # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.set_index.html
        charaDF.set_index("CHARA LOAD NUM", inplace = True)
        # https://stackoverflow.com/questions/10715965/add-one-row-in-a-pandas-dataframe
        # USE LOC INSTEAD OF ILOC TO ADD NEW ROW
    else:
        charaDic = {"CHAR NAME": CHAR_NAME, "DEX": DEX, "STR": STR, "EFF DEX": eff_DEX, "EFF STR": eff_STR,
                    "GUN TRAIN": GUN_TRAIN, "EFF GUN TRAIN": eff_GUN_TRAIN}
        charaDF.loc[CHAR_LOAD_NUM] = charaDic # USE LOC INSTEAD OF ILOC TO ADD NEW ROW
        # charaDF.loc[CHAR_LOAD_NUM][1] = DEX
        # charaDF.loc[CHAR_LOAD_NUM][2] = STR
        # charaDF.loc[CHAR_LOAD_NUM][3] = GUN_TRAIN
    
    
def gunDB(GUN_NAME, init, GUN_LOAD_NUM = 0):
    global gunDF
    
    if init == 1:
        gunDic = {"GUN NAME": GUN_NAME, "accuracy": accuracy, "recoil": recoil, "ROF": rateOfFire,
                 "Dmg Min": gunMin, "Dmg Max": gunMax, "Piercing": playerGun_Piercing, "Range": playerGun_Range,
                 "Type": weaponType, "Spread": weaponSpread, "GUN LOAD NUM": GUN_LOAD_NUM}
        gunDF = pd.DataFrame(gunDic, index = [0])
        # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.set_index.html
        gunDF.set_index("GUN LOAD NUM", inplace = True)
        # https://stackoverflow.com/questions/10715965/add-one-row-in-a-pandas-dataframe
        # USE LOC INSTEAD OF ILOC TO ADD NEW ROW
    else:
        
        gunDic = {"GUN NAME": GUN_NAME, "accuracy": accuracy, "recoil": recoil, "ROF": rateOfFire,
                 "Dmg Min": gunMin, "Dmg Max": gunMax, "Piercing": playerGun_Piercing, "Range": playerGun_Range,
                 "Type": weaponType, "Spread": weaponSpread}
        gunDF.loc[GUN_LOAD_NUM] = gunDic

        
    
    
def gun_MEN_DMG (enemyGun_Armor, hit_Counter, close_Miss_Counter):
    # Do I do flat damage modified by how well it was hit?
    # Or do I assume the damage handles how well it was hit like this one?
    # Note: Make armor/playerGun_Piercing calculation separate, so we can see how it increases/decreases damage.
    # Having playerGun_Piercing get affected by range.
    men_DMG_Counter = hit_Counter + close_Miss_Counter
    singleMen_DMG = 0
    totalMen_DMG = 0
    armorMod = ((1 + enemyGun_Armor / 10) - (playerGun_Piercing / 10))
    
    for men_Attempt in range (0, men_DMG_Counter):
        noMod_DMG = random.randint(int(gunMin / 3), int(gunMax / 3))
        singleMen_DMG = noMod_DMG / armorMod
        totalMen_DMG += singleMen_DMG
        
    return round(totalMen_DMG)

checkCharaDB = input("Do you have a charaDB.csv file in the same directory? (1 is yes, 0 is no): ")
checkGunDB = input("Do you have a gunDB.csv file in the same directory? (1 is yes, 0 is no): ")

if checkCharaDB == "1":
    charaDF = pd.read_csv('charaDB.csv')
    print ("Characters loaded!")
    print (charaDF)

if checkGunDB == "1":
    gunDF = pd.read_csv('gunDB.csv')
    print ("Guns loaded!")
    print (gunDF)

print ("Initialization Completed!")


 

# Create Character

In [ ]:
createShooterStats()
print (charaDF)

# Save and Overrwrite CharaDB

In [ ]:
charaDF.to_csv("charaDB.csv")
print ("Saved!")

# Load Character - Don't forget to do this first before initiating action!

In [ ]:
charaLoadActive = "1"
while charaLoadActive == "1":
        loadShooterStats()
        charaLoadActive = input("1 to continue, leave blank to cancel.")
clear_output()
print (charaDF)

# Create Gun

In [ ]:
createGunStats() 

# Save and Overrwrite GunDB

In [ ]:
gunDF.to_csv("gunDB.csv")
print ("Saved!")

# Load Gun -  Don't forget to do this first before initiating action!

In [ ]:
gunLoadActive = "1"
while gunLoadActive == "1":
        loadGunStats()
        gunLoadActive = input("1 to continue, leave blank to cancel.")
clear_output()
print (gunDF)

# Shoot Gun

In [ ]:
doneChecker = 0
counter = 1

while doneChecker != "2":
    # Must be "and" keyword in "while" so all must be true.
    if doneChecker != "1":
        if rateOfFire >= 8:
            bullet = input ("Firing Mode - Aimed: 0, Conserve: 1 - Control: 2 - Aggro: 3?: ")
        else:
            bullet = "0"
        bullet = fireRate(bullet)
        print ("Distance Key - Close: 0, Short: 1, Medium: 2. If >= 100 meters, enter distance in meters!")
        distance = int(input ("Distance [0-2 or Manual]: "))
        print ("Cover Key - Flanked: 0, Moving Target: 1, Aggro Cover: 2, Normal Cover: 3, Great Cover: 4, 5: Hunker." +
        " Enter Cover Value if GM instructs.")
        cover = int(input ("Cover [0-4 or Manual]: "))
        if cover == "":
           cover = 0
        enemyGun_Armor = int(input ("Target's BALLISTIC armor?: "))
    clear_output()
    
    bulletCalculation(bullet, eff_STR, eff_DEX, cover, distance, enemyGun_Armor)

    doneChecker = input("enter to repeat, 1 to continue, 2 to cancel")
    
    if counter == 2:
        clear_output()
    counter += 1
    

In [3]:
import nbinteract as nbi
nbi.publish('Users/Won/TAP', 'TAPS.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'TAPS.ipynb'.
Converting notebook...



Successfully converted!

<a href="TAPS.html" target="_blank" download>Click to download your webpage.</a>

To host your webpage, see the documentation:
<a href="https://www.nbinteract.com/tutorial/tutorial_publishing.html"
        target="_blank">
    https://www.nbinteract.com/tutorial/tutorial_publishing.html
</a>
